In [2]:
import pandas as pd
import os

In [3]:
with open('issues.csv') as f:
    data = pd.read_csv(f)

In [4]:
data.head()

,document_name,pipeline_highlight,correction,quality,supposed_cause,context,correct_label,possible_solution,comments
0,1919_Horwood_blackthorn_Cacopsylla_pruni_UK_tr...,NaN,5 ion,false_negative,"text_extraction_issue,unknown_lexeme","Blackberry, Blackthorn, Blackthorn-May, Buckth...",habitat,"add_pattern,taxon_autocorrector",NaN
1,1919_Horwood_blackthorn_Cacopsylla_pruni_UK_tr...,NaN,7: detule,false_negative,text_extraction_issue,"Otiorhynchus picipes, Monochetus sulcatus, Mag...",taxon,NaN,NaN
2,1955_Tamanini_Cacopsylla_pruni_Italie_premièr...,Acer,Acer campestris,too_narrow,unknown_lexeme,"Trentino, Garniga, on Acer campestris L., VII-...",taxon,add_lexeme,NaN
3,1955_Tamanini_Cacopsylla_pruni_Italie_premièr...,Acer,Acer campestris L. VII-1945 5,too_narrow,unknown_lexeme,"am. Psyiiiae Haupt. zarni is La, VIL-1945 5 # ...",taxon,add_lexeme,NaN
4,2020_Gjonov_New_records_of_Hemiptera_from_the_...,NaN,adelgid,false_negative,unknown_lexeme,The Sternorryncha of the Maltese Islands is r...,habitat,add_lexeme,NaN


In [5]:
data['possible_solution'] = list(map(lambda x: tuple(str(x).split(',')), data['possible_solution']))

In [6]:
set(data['possible_solution'])

{('add_lexeme',),
 ('add_pattern',),
 ('add_pattern', 'taxon_autocorrector'),
 ('case_sensitivity', 'common_names_detection'),
 ('case_sensitivity', 'common_names_detection', 'concept_addition'),
 ('case_sensitivity', 'concept_addition'),
 ('common_names_detection',),
 ('concept_addition', 'desambiguation'),
 ('discontinued_entity_treatement',),
 ('encoding_compatibility',),
 ('foreign_pattern_translation',),
 ('nan',),
 ('taxon_autocorrector',)}

In [10]:
data['problem'] = tuple(zip(data['quality'],data['supposed_cause'],data['correct_label']))
set(data['problem'])

{('false_neagative', 'unknown_lexeme', 'taxon'),
 ('false_negative', 'anaphora', 'habitat'),
 ('false_negative', 'anaphora', 'taxon'),
 ('false_negative', 'case_sensitivity', 'habitat'),
 ('false_negative', 'encoding', 'habitat'),
 ('false_negative', 'foreign_lexeme', 'habitat'),
 ('false_negative', 'foreign_lexeme', 'taxon'),
 ('false_negative', 'morphological variation', 'taxon'),
 ('false_negative', 'morphological_variation', 'taxon'),
 ('false_negative', 'outdated_term', 'habitat'),
 ('false_negative', 'pattern_recognition', 'habitat'),
 ('false_negative', 'pattern_recognition,anaphora', 'habitat'),
 ('false_negative', 'polysemy', 'habitat'),
 ('false_negative', 'text_extraction_issue', 'habitat'),
 ('false_negative', 'text_extraction_issue', 'taxon'),
 ('false_negative', 'text_extraction_issue', nan),
 ('false_negative', 'text_extraction_issue,unknown_lexeme', 'habitat'),
 ('false_negative', 'unknown_lexeme', 'habitat'),
 ('false_negative', 'unknown_lexeme', 'taxon'),
 ('false_neg